In [1]:
from create_daily_csv import xarray2df,check_data_gap,readnetcdf_in_shp_db,get_discharge_from_DB
from create_daily_csv import spatial_stats_daily_input
import pandas as pd
import numpy as np
import os
from matplotlib import pyplot as plot
import rioxarray as xr
import datetime
import psycopg2
from geopandas import GeoSeries

import pandas as pd
import numpy as np
import xarray as xr
import geopandas as gpd
from shapely.geometry import Point, Polygon
import matplotlib.pyplot as plt
import psycopg2
import shapely.wkb as wkb
from geopandas import GeoSeries

In [2]:
LIST =['ADO_DSC_CH03_0075',
     'ADO_DSC_AT31_0254',
     'ADO_DSC_ITC1_0072',
     'ADO_DSC_ITC1_0020',
     'ADO_DSC_CH07_0147',
     'ADO_DSC_AT31_0206',
     'ADO_DSC_ITH1_0012',
     'ADO_DSC_AT12_0280',
     'ADO_DSC_CH07_0100',
     'ADO_DSC_CH05_0201',
     'ADO_DSC_SI03_0148',
     'ADO_DSC_ITC1_0037',
     'ADO_DSC_FRK2_0042',
     'ADO_DSC_CH04_0011',
     'ADO_DSC_ITH2_0035',
     'ADO_DSC_SI03_0033',
     'ADO_DSC_FRK2_0041',
     'ADO_DSC_ITH5_0006',
     'ADO_DSC_CH07_0006']

tif_fileName =  r'C:\Users\mmazzolini\OneDrive - Scientific Network South Tyrol\DEM_EU\DEM.tif'

In [3]:
#
table=pd.DataFrame(data=None)
#dem=xr.open_rasterio(tif_fileName)
# Open the shape file and reproject it to the MESCAN-Surfex grid (unit=meters)

conn = psycopg2.connect(host="10.8.244.31",
                   database="climate_data",
                   user="ado_user",
                   password="hydro#ado",
                   port=5432)
cur = conn.cursor()

In [4]:
for STAT_CODE in LIST:
   

        
    # get the metadata
    query = f"""
            SELECT "geom" FROM "hydrology"."catchment_area" WHERE "id_station" = '{STAT_CODE}'    
            """
    df = pd.read_sql_query(query, conn)
    

    
    shp=GeoSeries(wkb.loads(df.geom[0], hex=True))
    shp=shp.set_crs("EPSG:4326")
    
    
    dem_catch=xr.open_rasterio(tif_fileName).rio.clip(shp[0],from_disk=True)

    h=np.array(dem_catch)
    h = h[h!=-99999.]
    
    h_mean=h.mean()
    
    update = {}
    update['name'] = STAT_CODE
    update['mean_h']=h_mean
    
    table = table.append(update,ignore_index=True)
    
# close the connection when finished
cur.close()
conn.close()   

In [5]:
table

,name,mean_h
0,ADO_DSC_CH03_0075,1063.146606
1,ADO_DSC_AT31_0254,841.597412
2,ADO_DSC_ITC1_0072,1531.825928
3,ADO_DSC_ITC1_0020,1255.702393
4,ADO_DSC_CH07_0147,1787.619263
5,ADO_DSC_AT31_0206,872.910583
6,ADO_DSC_ITH1_0012,1796.362549
7,ADO_DSC_AT12_0280,823.516418
8,ADO_DSC_CH07_0100,1526.887695
9,ADO_DSC_CH05_0201,914.721069


In [6]:
table.to_csv('id_stat_avgh2.csv')